In [1]:
import pandas as pd

import numpy as np

In [2]:

# srcDataFrame = pd.read_csv('optionsDX/aapl_2022/aapl_eod_2022q4-86egbl/aapl_eod_202210.txt')
srcDataFrame = pd.read_csv('tslaReduced.csv')

df = pd.DataFrame(srcDataFrame)
df.columns = df.columns.str.strip()

/var/folders/p_/r9zh86p106b8yh2ly8zzjbw00000gn/T/ipykernel_25237/155347403.py:2: DtypeWarning: Columns (4,5,6,8,9,10) have mixed types. Specify dtype option on import or set low_memory=False.
  srcDataFrame = pd.read_csv('tslaReduced.csv')


In [3]:
# df['[QUOTE_DATE]'] = df.get('[QUOTE_DATE]').str.strip()
# # df['[QUOTE_DATE]']

In [4]:
dayGroups = df.groupby('[QUOTE_DATE]')

In [5]:
from OptionPlayTranslator import *
# from utils.options_utils import *

In [6]:

opt = OPT('TSLA', dayGroups)
print(opt.ticker)
opt.find_plays(' 2019-10-01', ' 2020-01-16',200)

TSLA
TSLA on  2019-10-01 predicted to be 200 on  2020-01-16
Baseline (shares): 109.965% P/L:$2099.  Option PnL: -100.0% P/L: $0. Result for options:$-2099.65



(2099.648348053647, 0.0, -2099.648348053647)

In [7]:
predictions = pd.read_csv('tsla_predictions.csv')
x = predictions.iloc[30]

In [8]:
predictions.iloc[0]

Date    2021-03-05
Pred      679.9309
Name: 0, dtype: object

In [9]:
results = []
for i in range(len(predictions)):  
    pred = predictions.iloc[i]
    play = (opt.find_plays(' 2021-03-04',' '+pred['Date'],float(pred['Pred']),'ATM',100, False, False))
    if play is not None:
        results.append(play)


In [10]:
results

[(961.1517735059922, 900.1851851851852, -60.96658832080698),
 (905.6543070859809, 919.8952879581152, 14.240980872134287),
 (1084.8709080672404, 1092.3211169284468, 7.45020886120642),
 (1074.9939676666936, 900.0, -174.9939676666936),
 (1125.392101664924, 1160.0698080279233, 34.67770636299929),
 (1116.1264377061048, 1137.8708551483421, 21.744417442237364),
 (1138.8080109386312, 1120.8365809301117, -17.97143000851952),
 (1088.8281187163195, 1047.4408937386333, -41.387224977686174),
 (1128.512828762165, 1100.8313847752663, -27.681443986898785),
 (1051.025496662109, 984.0478046245778, -66.97769203753114),
 (1054.0014477599937, 966.7705897635749, -87.23085799641876),
 (1077.7768840987694, 1011.0141766630317, -66.76270743573775),
 (1065.2618032655032, 993.2388222464558, -72.02298101904739),
 (1013.6089439395158, 937.5136314067612, -76.09531253275463),
 (1029.5664763130378, 941.679389312977, -87.88708700006077),
 (995.4636853534948, 900.0, -95.46368535349484),
 (1111.5257781709965, 1016.972318